
<img src="https://gitlab.com/bivl2ab/academico/cursos-uis/ai/ai-2-uis-student/-/raw/master/imgs/Banner_IA2_1.png" style="width:1500px;heigth:100px">

In [14]:
#@title 1. MONTAR EL DRIVE  (If you are in in colab){ display-mode: "form" }
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/Proyecto-IA2')
print(os.getcwd())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/.shortcut-targets-by-id/1PQzbUvOzB5hWnfzPk0QwYiRx4vW7ma3N/Proyecto-IA2


In [15]:
#@title import libs
import os
import cv2
from PIL import Image
import tensorflow as tf
import numpy as np
import sys
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from PIL import Image

from tensorflow import keras
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, GlobalMaxPooling2D
from tensorflow.keras.models import Model

In [ ]:
#Import dataset image from API Kaagle
!pip install -q kaggle

In [ ]:
from google.colab import files

files.upload()

KeyboardInterrupt: ignored

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d ashishmotwani/tomato

In [ ]:
!unzip tomato

In [ ]:
!ls

In [16]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        './train',
        target_size=(32, 32),
        batch_size=32,
        class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
        './valid',
        target_size=(32, 32),
        batch_size=32,
        class_mode='categorical')


Found 25851 images belonging to 11 classes.
Found 6683 images belonging to 11 classes.


In [17]:
from glob import glob
folders = glob("/content/drive/.shortcut-targets-by-id/1PQzbUvOzB5hWnfzPk0QwYiRx4vW7ma3N/Proyecto-IA2/train/*")
folders

['/content/drive/.shortcut-targets-by-id/1PQzbUvOzB5hWnfzPk0QwYiRx4vW7ma3N/Proyecto-IA2/train/Bacterial_spot',
 '/content/drive/.shortcut-targets-by-id/1PQzbUvOzB5hWnfzPk0QwYiRx4vW7ma3N/Proyecto-IA2/train/Early_blight',
 '/content/drive/.shortcut-targets-by-id/1PQzbUvOzB5hWnfzPk0QwYiRx4vW7ma3N/Proyecto-IA2/train/Late_blight',
 '/content/drive/.shortcut-targets-by-id/1PQzbUvOzB5hWnfzPk0QwYiRx4vW7ma3N/Proyecto-IA2/train/Leaf_Mold',
 '/content/drive/.shortcut-targets-by-id/1PQzbUvOzB5hWnfzPk0QwYiRx4vW7ma3N/Proyecto-IA2/train/Septoria_leaf_spot',
 '/content/drive/.shortcut-targets-by-id/1PQzbUvOzB5hWnfzPk0QwYiRx4vW7ma3N/Proyecto-IA2/train/Spider_mites Two-spotted_spider_mite',
 '/content/drive/.shortcut-targets-by-id/1PQzbUvOzB5hWnfzPk0QwYiRx4vW7ma3N/Proyecto-IA2/train/Target_Spot',
 '/content/drive/.shortcut-targets-by-id/1PQzbUvOzB5hWnfzPk0QwYiRx4vW7ma3N/Proyecto-IA2/train/Tomato_Yellow_Leaf_Curl_Virus',
 '/content/drive/.shortcut-targets-by-id/1PQzbUvOzB5hWnfzPk0QwYiRx4vW7ma3N/Proyecto-

In [18]:
def get_model():
    
    base_model = MobileNet(input_shape=(32,32,3), include_top=False)
    '''
    for layers in base_model.layers[:76]:
        layers.trainable = False
    for layers in base_model.layers[76:]:
        layers.trainable = True
    ''' 
    base_model.trainable = False
    model = tf.keras.Sequential([base_model, 
                                        tf.keras.layers.GlobalAveragePooling2D(),
                                        tf.keras.layers.Dense(11, activation='softmax')],
                                        name="model")
    
    return model

In [19]:
model = get_model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenet_1.00_224 (Functio  (None, 1, 1, 1024)       3228864   
 nal)                                                            
                                                                 
 global_average_pooling2d_2   (None, 1024)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_3 (Dense)             (None, 11)                11275     
                                                                 
Total params: 3,240,139
Trainable params: 11,275
Non-trainable params: 3,228,864
_________________________________________________________________


In [20]:
learning_rate= 0.1
opt = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9)
model.compile(optimizer=opt, loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
history = model.fit(train_generator, validation_data=test_generator, epochs=100, verbose=1, batch_size=32)

Epoch 1/100
808/808 [==============================] - 211s 258ms/step - loss: 2.3959 - accuracy: 0.1359 - val_loss: 2.3797 - val_accuracy: 0.1496
Epoch 2/100
808/808 [==============================] - 193s 239ms/step - loss: 2.3949 - accuracy: 0.1406 - val_loss: 2.4035 - val_accuracy: 0.1683
Epoch 3/100
808/808 [==============================] - 191s 236ms/step - loss: 2.3806 - accuracy: 0.1448 - val_loss: 2.3718 - val_accuracy: 0.1559
Epoch 4/100
808/808 [==============================] - 192s 237ms/step - loss: 2.3921 - accuracy: 0.1457 - val_loss: 2.5813 - val_accuracy: 0.1425
Epoch 5/100
808/808 [==============================] - 191s 236ms/step - loss: 2.3901 - accuracy: 0.1418 - val_loss: 2.3897 - val_accuracy: 0.1255
Epoch 6/100
808/808 [==============================] - 193s 239ms/step - loss: 2.3685 - accuracy: 0.1491 - val_loss: 2.3285 - val_accuracy: 0.1561
Epoch 7/100
808/808 [==============================] - 199s 246ms/step - loss: 2.3755 - accuracy: 0.1518 - val_loss: 2

In [10]:
def get_model_02():
    
    base_model = MobileNet(input_shape=(224,224,3), include_top=False)
    '''
    for layers in base_model.layers[:76]:
        layers.trainable = False
    for layers in base_model.layers[76:]:
        layers.trainable = True
    ''' 
    base_model.trainable = False
    model = tf.keras.Sequential([base_model, 
                                        tf.keras.layers.GlobalAveragePooling2D(),
                                        tf.keras.layers.Dense(256, activation='relu'),
                                        tf.keras.layers.Dense(11, activation='softmax')],
                                        name="model")
    
    return model

In [11]:
model_02 = get_model_02()
model_02.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenet_1.00_224 (Functio  (None, 7, 7, 1024)       3228864   
 nal)                                                            
                                                                 
 global_average_pooling2d_1   (None, 1024)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_1 (Dense)             (None, 256)               262400    
                                                                 
 dense_2 (Dense)             (None, 11)                2827      
                                                                 
Total params: 3,494,091
Trainable params: 265,227
Non-trainable params: 3,228,864
_________________________________________________________________


In [12]:
model_exception.compile(optimizer=opt, loss='categorical_crossentropy',metrics=['accuracy'])
history_exception = model_exception.fit(train_generator, validation_data=test_generator, epochs=20, verbose=1, batch_size=32)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-12-0e7a847d445d>", line 1, in <module>
    model_exception.compile(optimizer=opt, loss='categorical_crossentropy',metrics=['accuracy'])
NameError: name 'model_exception' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py", line 2040, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/IPython/core/ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_

NameError: ignored